In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', use_cudnn_on_gpu=False)


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def charToNumpyArray(target_dataset):
    array = []
    for char in target_dataset:
        target_array = np.zeros(26)
        target_array[ord(char[0]) - 97] = 1
        array.append(target_array)
    return np.array(array)


def numpyArrayToChar(array_target):
    array = []
    for a in array_target:
        array.append(chr(a + 97))
    return array

In [3]:
print("Load Dataset")
train_data = np.array(pd.read_csv("train-data.csv", header=None))
train_target = np.array(pd.read_csv("train-target.csv", header=None))
test_data = np.array(pd.read_csv("test-data.csv", header=None))
test_target = np.array(pd.read_csv("test-target.csv", header=None))
print("Data loaded...")

print("Data Preparation Phase")

train_target_array = charToNumpyArray(train_target)
test_target_array = charToNumpyArray(test_target)

x = tf.placeholder(tf.float32, [None, 128])
y = tf.placeholder(tf.float32, [None, 26])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

#Reshape of the image into 16x8 matrices
x_image = tf.reshape(x, [-1, 16, 8, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([4 * 2 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 4*2*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 26])
b_fc2 = bias_variable([26])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_hat = tf.nn.softmax(y_conv)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_hat), reduction_indices=[1]))

predicted_y = tf.argmax(y_hat, 1)
real_y = tf.argmax(y, 1)
correct_prediction = tf.equal(predicted_y, real_y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()
init = tf.global_variables_initializer()

Load Dataset
Data loaded...
Data Preparation Phase


In [4]:
def run_train_batch(sess, train_x, train_y, opt, batch):
    sess.run(init)
    for epoch in range(10):
        total_batch = int(train_x.shape[0] / batch)
        for i in range(total_batch):
            batch_x = train_x[i*batch:(i+1)*batch]
            batch_y = train_y[i*batch:(i+1)*batch]
            sess.run(opt, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.9})

def cross_validation_over_batch(sess, learning_rate, train_data, train_target, batch_size):
    accuracy_list = []
    kf = KFold(n_splits=3)
    sess.run(init)
    #Train with gradient Descent Optimizer
    opt = tf.train.GradientDescentOptimizer(learning_rate=0.03).minimize(cross_entropy)
    i = 0
    for i_t, i_v in kf.split(train_data, train_target):
        print("\t\tLearning rate = " + str(learning_rate) + " N. fold = " + str(i))
        train_in = train_data[i_t]
        train_out = train_target[i_t]
        
        run_train_batch(sess, train_in, train_out, opt, batch_size)
        acc = sess.run(accuracy, feed_dict={x: train_data[i_v], y: train_target[i_v], keep_prob: 0.9})
        print("\t\tAccuracy: " + str(acc) + " N. fold = " + str(i) + '\n')
        accuracy_list.append(acc)
        i += 1
    return np.mean(accuracy_list)


In [5]:
#Cross Validation over batch size
print("Cross Validation with different learning rates with GradientDescentOptimizer")
learning_rate = 0.03
accuracy_scores = []
batches = [400, 500, 600]

for i in batches:
    print("\tBatch " + str(i))
    accuracy_out = cross_validation_over_batch(sess, learning_rate, train_data, train_target_array, i)
    accuracy_scores.append(accuracy_out)
    print("\tMean Accuracy = " + str(accuracy_out) + " batch number: " + str(i))

print('Accuracy scores: \n' + str(accuracy_scores))

Cross Validation with different learning rates with GradientDescentOptimizer
	Batch 400
		Learning rate = 0.03 N. fold = 0
		Accuracy: 0.8359819 N. fold = 0

		Learning rate = 0.03 N. fold = 1
		Accuracy: 0.8441073 N. fold = 1

		Learning rate = 0.03 N. fold = 2
		Accuracy: 0.8387143 N. fold = 2

	Mean Accuracy = 0.8396012 batch number: 400
	Batch 500
		Learning rate = 0.03 N. fold = 0
		Accuracy: 0.82907885 N. fold = 0

		Learning rate = 0.03 N. fold = 1
		Accuracy: 0.8308765 N. fold = 1

		Learning rate = 0.03 N. fold = 2
		Accuracy: 0.830517 N. fold = 2

	Mean Accuracy = 0.83015746 batch number: 500
	Batch 600
		Learning rate = 0.03 N. fold = 0
		Accuracy: 0.81642336 N. fold = 0

		Learning rate = 0.03 N. fold = 1
		Accuracy: 0.82066584 N. fold = 1

		Learning rate = 0.03 N. fold = 2
		Accuracy: 0.81476957 N. fold = 2

	Mean Accuracy = 0.81728625 batch number: 600
Accuracy scores: 
[0.8396012, 0.83015746, 0.81728625]


In [6]:
print("Training with the best accuracy score index")

best_batch = np.array(accuracy_scores).argmax()
print("Best batch number selected = " + str(best_batch))

#Training phase
print("TRAINING using Gradient Descent Optimizer")
train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
run_train_batch(sess, train_data, train_target_array, train_step, batches[best_batch])

print("Testing")
accuracy, array_test_hat = sess.run([accuracy, predicted_y], feed_dict={x: test_data, y: test_target_array, keep_prob: 0.9})

Training with the best accuracy score index
Best batch number selected = 0
TRAINING using Gradient Descent Optimizer
Testing


In [7]:
print("\tData target converting")
test_hat = numpyArrayToChar(array_test_hat)

print("Accuracy: " + str(accuracy))

print("Save predicted labels")
f = open("labels.txt", 'w')
for out in test_hat:
    f.write(out + "\n")
f.close()

	Data target converting
Accuracy: 0.8656888
Save predicted labels
